#### dataset info : http://archive.ics.uci.edu/ml/datasets/Bank+Marketing#
#### http://dx.doi.org/10.1016/j.dss.2014.03.001
#### http://www3.dsi.uminho.pt/pcortez/bib/2014-dss.txt

Note: we are using the file that is 10% of the original dataset


## lab info: 

"This assignment uses data(do see attachment) from the Bank Marketing data set from the UCI ML Archive (http://archive.ics.uci.edu/ml/datasets/Bank+Marketing).

The data set has 20 feature columns plus one result column and we need to do some work to get it ready for further processing.

1. Reference the bank-additional-names.txt file for column types and what the names mean.

2. Make the following changes:

Change column names to remove abbreviations, capitalize, add spaces, and generally make the names more "meaningful" to casual readers.
Change column types to match the associated feature types.
Replace word separators in strings like "-" or "." with spaces.
Do any values need to be replaced with NaN or are they better the way they are?
#8 in the text file states,
3. Missing Attribute Values: There are several missing values in some categorical attributes, all coded with the "unknown" label. These missing values can be treated as a possible class label or using deletion or imputation techniques.

Decide what to do with these "unknown" values and defend your decision.
Deliverable: Upload your notebook."

## 1 -- setup
### load dataset & seperate into columns

In [1]:
# Here are packages we will import and will use throughout the lab(mostly). We will discuss them more while they are in use.

import os
import requests

import pandas as pd
import numpy as np


import re

In [2]:
## we must seperate this dataset into individual columns originally seperated by ';'
df = pd.read_csv('bank-additional.csv', sep=';')

In [3]:
# we see we havwe nice and clean dataset now
df.head

<bound method NDFrame.head of       age          job  marital          education default  housing     loan  \
0      30  blue-collar  married           basic.9y      no      yes       no   
1      39     services   single        high.school      no       no       no   
2      25     services  married        high.school      no      yes       no   
3      38     services  married           basic.9y      no  unknown  unknown   
4      47       admin.  married  university.degree      no      yes       no   
...   ...          ...      ...                ...     ...      ...      ...   
4114   30       admin.  married           basic.6y      no      yes      yes   
4115   39       admin.  married        high.school      no      yes       no   
4116   27      student   single        high.school      no       no       no   
4117   58       admin.  married        high.school      no       no       no   
4118   34   management   single        high.school      no      yes       no   

        c

## 2 -- making changes
#### checking shape and type

In [4]:
# checking type
print(type(df))

<class 'pandas.core.frame.DataFrame'>


In [5]:
# checking shape. we see we have 21 columns now and 4,119 rows of data
print(df.shape)

(4119, 21)


In [6]:
# we see some unknown and nonexistent data. we will adjust this later on
x = df.iloc[:,2:].values
x[:5,:]

array([['married', 'basic.9y', 'no', 'yes', 'no', 'cellular', 'may',
        'fri', 487, 2, 999, 0, 'nonexistent', -1.8, 92.89299999999999,
        -46.2, 1.3130000000000002, 5099.1, 'no'],
       ['single', 'high.school', 'no', 'no', 'no', 'telephone', 'may',
        'fri', 346, 4, 999, 0, 'nonexistent', 1.1, 93.994, -36.4, 4.855,
        5191.0, 'no'],
       ['married', 'high.school', 'no', 'yes', 'no', 'telephone', 'jun',
        'wed', 227, 1, 999, 0, 'nonexistent', 1.4, 94.465, -41.8, 4.962,
        5228.1, 'no'],
       ['married', 'basic.9y', 'no', 'unknown', 'unknown', 'telephone',
        'jun', 'fri', 17, 3, 999, 0, 'nonexistent', 1.4, 94.465, -41.8,
        4.959, 5228.1, 'no'],
       ['married', 'university.degree', 'no', 'yes', 'no', 'cellular',
        'nov', 'mon', 58, 1, 999, 0, 'nonexistent', -0.1, 93.2, -42.0,
        4.191, 5195.8, 'no']], dtype=object)

#### aggregate mean average 

In [7]:
#here is a simple way to see variables with intergers. We know that the mean of pdays is skewed due to all the '999' values. we will adjust this later
df.agg("mean")

age                 40.113620
duration           256.788055
campaign             2.537266
pdays              960.422190
previous             0.190337
emp.var.rate         0.084972
cons.price.idx      93.579704
cons.conf.idx      -40.499102
euribor3m            3.621356
nr.employed       5166.481695
dtype: float64

#### Transpose

In [8]:
## to get a better perspective of the dataset
df.T
# we can see pdays has many '999' values,  poutcome has nonexixtent/failures, and there could be other unknown data we might need to sort out

,0,1,2,3,4,5,6,7,8,9,...,4109,4110,4111,4112,4113,4114,4115,4116,4117,4118
age,30,39,25,38,47,32,32,41,31,35,...,63,53,30,31,31,30,39,27,58,34
job,blue-collar,services,services,services,admin.,services,admin.,entrepreneur,services,blue-collar,...,retired,housemaid,technician,technician,admin.,admin.,admin.,student,admin.,management
marital,married,single,married,married,married,single,single,married,divorced,married,...,married,divorced,married,single,single,married,married,single,married,single
education,basic.9y,high.school,high.school,basic.9y,university.degree,university.degree,university.degree,university.degree,professional.course,basic.9y,...,high.school,basic.6y,university.degree,professional.course,university.degree,basic.6y,high.school,high.school,high.school,high.school
default,no,no,no,no,no,no,no,unknown,no,unknown,...,no,unknown,no,no,no,no,no,no,no,no
housing,yes,no,yes,unknown,yes,no,yes,yes,no,no,...,no,unknown,no,yes,yes,yes,yes,no,no,yes
loan,no,no,no,unknown,no,no,no,no,no,no,...,no,unknown,yes,no,no,yes,no,no,no,no
contact,cellular,telephone,telephone,telephone,cellular,cellular,cellular,cellular,cellular,telephone,...,cellular,telephone,cellular,cellular,cellular,cellular,telephone,cellular,cellular,cellular
month,may,may,jun,jun,nov,sep,sep,nov,nov,may,...,oct,may,jun,nov,nov,jul,jul,may,aug,nov
day_of_week,fri,fri,wed,fri,mon,thu,mon,mon,tue,thu,...,wed,fri,fri,thu,thu,thu,fri,mon,fri,wed


#### stack by 'job'

In [9]:
# an interesting perspective by job type
st = df.pivot_table(index='job').stack
st
# we are mostly just testing the water right now to get differnt perspective

<bound method DataFrame.stack of                      age  campaign  cons.conf.idx  cons.price.idx    duration  \
job                                                                             
admin.         38.240119  2.642292     -40.014723       93.527008  261.871542   
blue-collar    39.265837  2.432127     -41.571493       93.669430  261.852941   
entrepreneur   42.202703  2.216216     -40.839865       93.614682  249.202703   
housemaid      45.672727  2.463636     -39.324545       93.658536  229.663636   
management     42.429012  2.432099     -40.591667       93.483056  246.799383   
retired        60.873494  2.397590     -39.034337       93.487506  311.789157   
self-employed  40.679245  2.817610     -40.323899       93.575283  254.924528   
services       38.513995  2.844784     -41.532061       93.626059  232.529262   
student        26.695122  1.951220     -39.665854       93.448329  287.134146   
technician     38.622287  2.509407     -39.895948       93.575912  253.28654

In [10]:
# loan
st = df.pivot_table(index='loan').stack
st

<bound method DataFrame.stack of                age  campaign  cons.conf.idx  cons.price.idx    duration  \
loan                                                                      
no       40.163332  2.564049     -40.512213       93.582737  259.236787   
unknown  39.523810  2.580952     -40.207619       93.634800  243.923810   
yes      39.956391  2.395489     -40.479098       93.555732  246.487218   

         emp.var.rate  euribor3m  nr.employed       pdays  previous  
loan                                                                 
no           0.081188   3.612484  5165.858107  959.856972  0.190206  
unknown      0.228571   3.743962  5171.364762  961.190476  0.285714  
yes          0.081353   3.646675  5168.851128  963.147368  0.175940  >

In [11]:
# loan pivot_table
st = df.pivot_table(index='loan').stack
st

<bound method DataFrame.stack of                age  campaign  cons.conf.idx  cons.price.idx    duration  \
loan                                                                      
no       40.163332  2.564049     -40.512213       93.582737  259.236787   
unknown  39.523810  2.580952     -40.207619       93.634800  243.923810   
yes      39.956391  2.395489     -40.479098       93.555732  246.487218   

         emp.var.rate  euribor3m  nr.employed       pdays  previous  
loan                                                                 
no           0.081188   3.612484  5165.858107  959.856972  0.190206  
unknown      0.228571   3.743962  5171.364762  961.190476  0.285714  
yes          0.081353   3.646675  5168.851128  963.147368  0.175940  >

### rename
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rename.html

In [12]:
df.rename(columns={1: "Age"})

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,30,blue-collar,married,basic.9y,no,yes,no,cellular,may,fri,...,2,999,0,nonexistent,-1.8,92.893,-46.2,1.313,5099.1,no
1,39,services,single,high.school,no,no,no,telephone,may,fri,...,4,999,0,nonexistent,1.1,93.994,-36.4,4.855,5191.0,no
2,25,services,married,high.school,no,yes,no,telephone,jun,wed,...,1,999,0,nonexistent,1.4,94.465,-41.8,4.962,5228.1,no
3,38,services,married,basic.9y,no,unknown,unknown,telephone,jun,fri,...,3,999,0,nonexistent,1.4,94.465,-41.8,4.959,5228.1,no
4,47,admin.,married,university.degree,no,yes,no,cellular,nov,mon,...,1,999,0,nonexistent,-0.1,93.200,-42.0,4.191,5195.8,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4114,30,admin.,married,basic.6y,no,yes,yes,cellular,jul,thu,...,1,999,0,nonexistent,1.4,93.918,-42.7,4.958,5228.1,no
4115,39,admin.,married,high.school,no,yes,no,telephone,jul,fri,...,1,999,0,nonexistent,1.4,93.918,-42.7,4.959,5228.1,no
4116,27,student,single,high.school,no,no,no,cellular,may,mon,...,2,999,1,failure,-1.8,92.893,-46.2,1.354,5099.1,no
4117,58,admin.,married,high.school,no,no,no,cellular,aug,fri,...,1,999,0,nonexistent,1.4,93.444,-36.1,4.966,5228.1,no


In [13]:
df.T

,0,1,2,3,4,5,6,7,8,9,...,4109,4110,4111,4112,4113,4114,4115,4116,4117,4118
age,30,39,25,38,47,32,32,41,31,35,...,63,53,30,31,31,30,39,27,58,34
job,blue-collar,services,services,services,admin.,services,admin.,entrepreneur,services,blue-collar,...,retired,housemaid,technician,technician,admin.,admin.,admin.,student,admin.,management
marital,married,single,married,married,married,single,single,married,divorced,married,...,married,divorced,married,single,single,married,married,single,married,single
education,basic.9y,high.school,high.school,basic.9y,university.degree,university.degree,university.degree,university.degree,professional.course,basic.9y,...,high.school,basic.6y,university.degree,professional.course,university.degree,basic.6y,high.school,high.school,high.school,high.school
default,no,no,no,no,no,no,no,unknown,no,unknown,...,no,unknown,no,no,no,no,no,no,no,no
housing,yes,no,yes,unknown,yes,no,yes,yes,no,no,...,no,unknown,no,yes,yes,yes,yes,no,no,yes
loan,no,no,no,unknown,no,no,no,no,no,no,...,no,unknown,yes,no,no,yes,no,no,no,no
contact,cellular,telephone,telephone,telephone,cellular,cellular,cellular,cellular,cellular,telephone,...,cellular,telephone,cellular,cellular,cellular,cellular,telephone,cellular,cellular,cellular
month,may,may,jun,jun,nov,sep,sep,nov,nov,may,...,oct,may,jun,nov,nov,jul,jul,may,aug,nov
day_of_week,fri,fri,wed,fri,mon,thu,mon,mon,tue,thu,...,wed,fri,fri,thu,thu,thu,fri,mon,fri,wed


### Change column Names using df.rename()

In [14]:
df.rename(columns={'age': "Age", 'job': 'Employment Status', 'marital': 'Marital', 'education': "Education", 'housing': "Housing", 'loan': "Loan", 'contact': "Contact", 'month': "Month", 'default': "Default", 'day_of_week': "Day", 'campaign': "Campaign"})

,Age,Employment Status,Marital,Education,Default,Housing,Loan,Contact,Month,Day,...,Campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,30,blue-collar,married,basic.9y,no,yes,no,cellular,may,fri,...,2,999,0,nonexistent,-1.8,92.893,-46.2,1.313,5099.1,no
1,39,services,single,high.school,no,no,no,telephone,may,fri,...,4,999,0,nonexistent,1.1,93.994,-36.4,4.855,5191.0,no
2,25,services,married,high.school,no,yes,no,telephone,jun,wed,...,1,999,0,nonexistent,1.4,94.465,-41.8,4.962,5228.1,no
3,38,services,married,basic.9y,no,unknown,unknown,telephone,jun,fri,...,3,999,0,nonexistent,1.4,94.465,-41.8,4.959,5228.1,no
4,47,admin.,married,university.degree,no,yes,no,cellular,nov,mon,...,1,999,0,nonexistent,-0.1,93.200,-42.0,4.191,5195.8,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4114,30,admin.,married,basic.6y,no,yes,yes,cellular,jul,thu,...,1,999,0,nonexistent,1.4,93.918,-42.7,4.958,5228.1,no
4115,39,admin.,married,high.school,no,yes,no,telephone,jul,fri,...,1,999,0,nonexistent,1.4,93.918,-42.7,4.959,5228.1,no
4116,27,student,single,high.school,no,no,no,cellular,may,mon,...,2,999,1,failure,-1.8,92.893,-46.2,1.354,5099.1,no
4117,58,admin.,married,high.school,no,no,no,cellular,aug,fri,...,1,999,0,nonexistent,1.4,93.444,-36.1,4.966,5228.1,no


In [15]:
df.agg("mean")

age                 40.113620
duration           256.788055
campaign             2.537266
pdays              960.422190
previous             0.190337
emp.var.rate         0.084972
cons.price.idx      93.579704
cons.conf.idx      -40.499102
euribor3m            3.621356
nr.employed       5166.481695
dtype: float64

## Renaming columns for types to match feature types

In [16]:
#this will make the names easier to understand and match better with the .txt file of descriptions
df1 = df.rename(columns={'age': "Age of client", 'job': 'Employment Status', 'marital': 'Marital Status', 'education': "Education", 'housing': "Housing", 'loan': "Loan", 'contact': "Contact", 'month': "Month", 'default': "Default", 'day_of_week': "Day", 'campaign': "Contracts performed during Campaign", 'pdays': "Days Since Clinet contact", 'previous': "Contracts previously performed before this contract", 'poutcome': "Prev. Marketing Outcome", 'emp.var.rate': "Quarterly Employment Variation rate", 'cons.price.idx': "Monthly Consumer Price Index", 'cons.conf.idx': "Monthly Consumer Confidence Index", 'euribor3m': "Euribor 3 Month Rate - daily", 'nr.employed': "Number of Employees - quarterly", 'y': "Term Deposit?", 'duration': "Duration in seconds of last conversation"})
df1

,Age of client,Employment Status,Marital Status,Education,Default,Housing,Loan,Contact,Month,Day,...,Contracts performed during Campaign,Days Since Clinet contact,Contracts previously performed before this contract,Prev. Marketing Outcome,Quarterly Employment Variation rate,Monthly Consumer Price Index,Monthly Consumer Confidence Index,Euribor 3 Month Rate - daily,Number of Employees - quarterly,Term Deposit?
0,30,blue-collar,married,basic.9y,no,yes,no,cellular,may,fri,...,2,999,0,nonexistent,-1.8,92.893,-46.2,1.313,5099.1,no
1,39,services,single,high.school,no,no,no,telephone,may,fri,...,4,999,0,nonexistent,1.1,93.994,-36.4,4.855,5191.0,no
2,25,services,married,high.school,no,yes,no,telephone,jun,wed,...,1,999,0,nonexistent,1.4,94.465,-41.8,4.962,5228.1,no
3,38,services,married,basic.9y,no,unknown,unknown,telephone,jun,fri,...,3,999,0,nonexistent,1.4,94.465,-41.8,4.959,5228.1,no
4,47,admin.,married,university.degree,no,yes,no,cellular,nov,mon,...,1,999,0,nonexistent,-0.1,93.200,-42.0,4.191,5195.8,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4114,30,admin.,married,basic.6y,no,yes,yes,cellular,jul,thu,...,1,999,0,nonexistent,1.4,93.918,-42.7,4.958,5228.1,no
4115,39,admin.,married,high.school,no,yes,no,telephone,jul,fri,...,1,999,0,nonexistent,1.4,93.918,-42.7,4.959,5228.1,no
4116,27,student,single,high.school,no,no,no,cellular,may,mon,...,2,999,1,failure,-1.8,92.893,-46.2,1.354,5099.1,no
4117,58,admin.,married,high.school,no,no,no,cellular,aug,fri,...,1,999,0,nonexistent,1.4,93.444,-36.1,4.966,5228.1,no


We see we successfully changed the names and types to make more sense. We now have some manipulation to continue with the dataset

In [17]:
#checking aggregations of means with new names that are easier to understand and more meaningful
df1.agg("mean")
#this is an easy way to identify any quantifiable/numeric variables

Age of client                                            40.113620
Duration in seconds of last conversation                256.788055
Contracts performed during Campaign                       2.537266
Days Since Clinet contact                               960.422190
Contracts previously performed before this contract       0.190337
Quarterly Employment Variation rate                       0.084972
Monthly Consumer Price Index                             93.579704
Monthly Consumer Confidence Index                       -40.499102
Euribor 3 Month Rate - daily                              3.621356
Number of Employees - quarterly                        5166.481695
dtype: float64

pivots by various non-interger variables to give unique insight:

In [18]:
#pivot by 'Marital Status' -- unique perspective
df1.pivot_table(columns={'Marital Status'})
# having unknown data could be helpful for further analysis, so we will leave 

Marital Status,divorced,married,single,unknown
Age of client,45.107623,42.389398,33.209020,42.272727
Contracts performed during Campaign,2.542601,2.527700,2.555941,2.545455
Contracts previously performed before this contract,0.174888,0.173774,0.233304,0.090909
Days Since Clinet contact,972.255605,964.162216,947.338248,999.000000
Duration in seconds of last conversation,263.163677,256.289757,254.647875,336.272727
Euribor 3 Month Rate - daily,3.707177,3.728554,3.352657,3.855182
Monthly Consumer Confidence Index,-40.621973,-40.357114,-40.768257,-39.690909
Monthly Consumer Price Index,93.600547,93.599039,93.529971,93.537545
Number of Employees - quarterly,5170.145964,5170.724472,5155.756114,5174.409091
Quarterly Employment Variation rate,0.143274,0.172260,-0.128448,0.181818


In [19]:
#pivot by 'Prev. Marketing Outcome'
df1.pivot_table(columns={'Prev. Marketing Outcome'})
#having 'nonexistent outcomes' still have data that could possibly be useful, so we will leave

Prev. Marketing Outcome,failure,nonexistent,success
Age of client,40.400881,39.981266,42.478873
Contracts performed during Campaign,1.958150,2.641783,1.795775
Contracts previously performed before this contract,1.209251,0.000000,1.654930
Days Since Clinet contact,959.825991,999.000000,5.218310
Duration in seconds of last conversation,254.612335,254.561169,318.992958
Euribor 3 Month Rate - daily,1.652077,3.979500,1.031979
Monthly Consumer Confidence Index,-42.633040,-40.309736,-38.374648
Monthly Consumer Price Index,93.101676,93.648752,93.394979
Number of Employees - quarterly,5090.048238,5181.894039,5028.475352
Quarterly Employment Variation rate,-1.656167,0.394891,-2.037324


In [20]:
#pivot table by month
df1.pivot_table(columns={'Month'})
# we see no data for 2 months: Jan & Feb

Month,apr,aug,dec,jul,jun,mar,may,nov,oct,sep
Age of client,41.158140,41.405660,46.000000,39.008439,39.833962,41.895833,39.420174,40.340807,44.797101,43.296875
Contracts performed during Campaign,1.962791,2.705975,2.636364,3.085795,2.988679,1.958333,2.423803,1.856502,1.463768,1.703125
Contracts previously performed before this contract,0.372093,0.135220,0.681818,0.054852,0.084906,0.729167,0.171988,0.302691,0.637681,1.062500
Days Since Clinet contact,966.655814,955.224843,773.227273,982.282700,968.996226,772.125000,981.682148,941.091928,855.289855,673.140625
Duration in seconds of last conversation,291.293023,218.435535,324.318182,278.015471,245.035849,274.916667,259.260522,257.051570,271.449275,275.812500
Euribor 3 Month Rate - daily,1.362698,4.270874,0.830318,4.710661,4.221585,1.179729,3.245085,3.675067,1.170957,0.842047
Monthly Consumer Confidence Index,-46.286047,-35.659906,-33.586364,-42.419550,-41.567925,-42.716667,-40.705806,-41.513004,-34.624638,-34.492188
Monthly Consumer Price Index,93.118888,93.327670,92.714955,93.902000,94.233887,93.095042,93.515037,93.239848,93.178855,93.488062
Number of Employees - quarterly,5093.213488,5197.916038,5030.445455,5215.130098,5195.895283,5055.783333,5148.142816,5168.913004,5017.381159,4984.654687
Quarterly Employment Variation rate,-1.800000,0.732075,-2.872727,1.184529,0.651132,-1.800000,-0.204790,-0.420179,-2.446377,-1.998437


In [21]:
#pivot by Education Level
df1.pivot_table(columns={'Education'})
# we see unknown values in this output, but they contain data so we should leave it be

Education,basic.4y,basic.6y,basic.9y,high.school,illiterate,professional.course,university.degree,unknown
Age of client,47.657343,40.144737,39.231707,38.097720,42.000,40.207477,39.017405,42.826347
Contracts performed during Campaign,2.421911,2.649123,2.348432,2.630836,4.000,2.512150,2.583070,2.538922
Contracts previously performed before this contract,0.142191,0.140351,0.158537,0.206298,0.000,0.194393,0.207278,0.263473
Days Since Clinet contact,973.524476,985.881579,979.963415,958.022801,999.000,958.211215,947.900316,939.700599
Duration in seconds of last conversation,255.682984,259.000000,250.538328,258.534202,146.000,278.816822,247.707278,267.281437
Euribor 3 Month Rate - daily,3.828089,3.810469,3.722737,3.511732,0.834,3.701426,3.547132,3.410174
Monthly Consumer Confidence Index,-40.465501,-41.539912,-41.443554,-40.995765,-31.400,-40.127664,-39.830063,-39.487425
Monthly Consumer Price Index,93.666193,93.689114,93.640805,93.564314,92.201,93.599630,93.499109,93.637455
Number of Employees - quarterly,5174.093939,5176.600000,5173.182578,5163.212595,5076.200,5167.595140,5163.023180,5151.260479
Quarterly Employment Variation rate,0.291841,0.271053,0.183275,-0.002497,-2.900,0.163925,-0.009731,-0.074251


Using multiple columns in a pivot

In [22]:
#multiple pivots
df1.pivot_table(columns={'Education', 'Contact', 'Month', 'Housing', 'Day'})

                                 Education  Month  Housing  Contact    Day
Age of client                    basic.4y   apr    no       cellular   fri      45.0
                                                                       mon      41.0
                                                                       thu      45.0
                                                                       tue      56.5
                                                            telephone  tue      59.0
                                                                               ...  
Number of Employees - quarterly  unknown    oct    yes      telephone  tue    5195.8
                                            sep    no       cellular   thu    5017.5
                                                   yes      cellular   thu    4963.6
                                                                       tue    4963.6
                                                                       wed 

### Replace '999' values with NaN in 'pdays', now 'Days Since Client contact' column

In [23]:
# the value '999' is giving us an incorrect value while looking at column named 'Days since client contact'
# this will give us accurate numbers when we aggregate them and do further analysis later
df1.replace(999, np.nan)
# we now see NaN in 'Days since client contact' column

,Age of client,Employment Status,Marital Status,Education,Default,Housing,Loan,Contact,Month,Day,...,Contracts performed during Campaign,Days Since Clinet contact,Contracts previously performed before this contract,Prev. Marketing Outcome,Quarterly Employment Variation rate,Monthly Consumer Price Index,Monthly Consumer Confidence Index,Euribor 3 Month Rate - daily,Number of Employees - quarterly,Term Deposit?
0,30,blue-collar,married,basic.9y,no,yes,no,cellular,may,fri,...,2,NaN,0,nonexistent,-1.8,92.893,-46.2,1.313,5099.1,no
1,39,services,single,high.school,no,no,no,telephone,may,fri,...,4,NaN,0,nonexistent,1.1,93.994,-36.4,4.855,5191.0,no
2,25,services,married,high.school,no,yes,no,telephone,jun,wed,...,1,NaN,0,nonexistent,1.4,94.465,-41.8,4.962,5228.1,no
3,38,services,married,basic.9y,no,unknown,unknown,telephone,jun,fri,...,3,NaN,0,nonexistent,1.4,94.465,-41.8,4.959,5228.1,no
4,47,admin.,married,university.degree,no,yes,no,cellular,nov,mon,...,1,NaN,0,nonexistent,-0.1,93.200,-42.0,4.191,5195.8,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4114,30,admin.,married,basic.6y,no,yes,yes,cellular,jul,thu,...,1,NaN,0,nonexistent,1.4,93.918,-42.7,4.958,5228.1,no
4115,39,admin.,married,high.school,no,yes,no,telephone,jul,fri,...,1,NaN,0,nonexistent,1.4,93.918,-42.7,4.959,5228.1,no
4116,27,student,single,high.school,no,no,no,cellular,may,mon,...,2,NaN,1,failure,-1.8,92.893,-46.2,1.354,5099.1,no
4117,58,admin.,married,high.school,no,no,no,cellular,aug,fri,...,1,NaN,0,nonexistent,1.4,93.444,-36.1,4.966,5228.1,no


Aggregate without skewed '999' values:

In [24]:
## creating new df2 to be able to kshow new aggregate for mean of column 'Days since client contact'
df2 = df1.replace(999, np.nan)
df2.agg("mean")
#we see it went from a mean of 960.42 previously to an accurate and truely reflective number of the value that is now a mean of 5.86 days. A large diffference that '999' values caused

Age of client                                            40.113620
Duration in seconds of last conversation                256.607819
Contracts performed during Campaign                       2.537266
Days Since Clinet contact                                 5.862500
Contracts previously performed before this contract       0.190337
Quarterly Employment Variation rate                       0.084972
Monthly Consumer Price Index                             93.579704
Monthly Consumer Confidence Index                       -40.499102
Euribor 3 Month Rate - daily                              3.621356
Number of Employees - quarterly                        5166.481695
dtype: float64

## 3 -- Missing Attribute Values:

After running some various analysis and breifly learning this dataset, we were able to identify various "unknown" labels. In most cases, they were labels as 'unknown' or 'nonexistent' that was mentioned and discussed above in the workset. These cases of 'unknown' or 'nonexistent' in particular had numerous interger data in their 'unknown' or 'nonexistent' class with corresponding data - therfore: we will not change or tamper with this data as it could lead to further insight or analysis in the future. Without enough knowledge of the dataset and more information upon what these variables mean, we will conclude that they are valueable and we will not make any change or adjustment to the dataset on that front, other than take notes for now. There was however, 1 column that did require changes to be made. It was the 'pdays' column that was renamed to be 'Days since client contact' for clarification purposes. In that 'Days since client contact' column, there were many '999' values along with other much smaller numbers to represent days since last in contact with client. The 999 value was skewing the final results. To eliminate this skewed aggregation/imputation, we changed the '999' values to be 'NaN' values (non interger) to not skew the data. With this change, we were able to better represent the intigrity of the dataset and provide numbers that are not altered due to 'missing' or 'distorted' data. After there changes and note, we will be able to pass the dataset along for further data analysis if needed. 

## Conclusion
In summary, we were able to reference the bank-additional-names.txt file with the bank-additional.csv to make changes by removing unnesecarry abbreviations/spaces/capitalizations to more "meaningful" names to capture the reader and make it easier to understand. We changed column types to match the appropriate feature being analyzed and replaced seperators in strings. We dissected the dataset and decided to leave the 'unknown' and 'nonexistent' data, as they contained useful metrics and intergers for future analysis potentially. We did however make change to the 'pdays' column that was later chaged to 'Days since client contact' by changing all '999' values that were skewing the metrics with 'NaN' to be able to provide accurate and useful data for further evaluation. 

## References:
Chen, D. Y. (2019). Python Data Analysis. Pearson Addison Wesley Data & Analytics Series. In Pandas for Everyone.

Docs.Python.org, . (2020). string — Common string operations. In Docs.Python. Retrieved from https://docs.python.org/3/library/string.html#custom-string-formatting.

Tamang, A. (2018, February 3). Unicode, UTF-8, and ASCII encodings made easy. In Medium. Retrieved from https://medium.com/@apiltamang/unicode-utf-8-and-ascii-encodings-made-easy-5bfbe3a1c45a.

Temple.edu, . (n.d.). introduce the world of ASCII and Unicode. In Temple University. Retrieved from https://sites.temple.edu/lizhe/2018/09/16/introduce-the-world-of-ascii-and-unicode/.
